In [1]:
import numpy as np
import time
import json

from collections import Counter
from braket.circuits import Circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

## Create circuit for IonQ quantum processing unit

The following function uses AWS Braket to receives shot data from an IonQ device

In [ ]:

def bracket_circuit(inputs, weights, num_qubits):
    circuit = Circuit()
    # RUN MODEL HERE
    
    ########################################################################
    
    # RUN ADJOINT MODEL HERE

    return circuit


## For-loop over qubit range, collect data, & save as a JSON

In [ ]:
min_qubits = 2
max_qubits = 18
qubit_range = range(min_qubits,max_qubits + 2, 2)
final_result = {}


for num_qubits in qubit_range:
    qubit_res_all_attempts = []
    time_all_attempts = []

    QPU = AwsDevice('arn:aws:braket:::device/qpu/ionq/ionQdevice')
    weights = (np.pi/4)*np.ones(num_qubits)
    inputs = (np.pi/4)*np.ones(num_qubits//2)
    
    for i in range(10):
        start = time.time()
        circ = bracket_circuit(inputs, weights, num_qubits)
        task = QPU.run(circ, shots=1000)
        res = task.result()
        total_time = time.time() - start
        time_all_attempts.append(total_time)
        qubit_res_all_attempts.append(res.measurement_counts)
        
    counter_list = qubit_res_all_attempts
    counter_list_sum = sum(counter_list, Counter())
    counter_list_mean = Counter({key : value/10 for key, value in counter_list_sum.items()})
    counter_list_mean_sq = Counter({key : value**2 for key, value in counter_list_mean.items()})


    counter_list_sq = [Counter({key : value**2 for key, value in elem.items()}) for elem in counter_list]
    counter_list_sq_sum = sum(counter_list_sq, Counter())
    counter_list_sq_mean = Counter({key : value/10 for key, value in counter_list_sq_sum.items()})
    counter_list_std = Counter({key : value**0.5 for key, value in (counter_list_sq_mean - counter_list_mean_sq).items()})

    print(f'For {num_qubits} qubits execution is finished')
    final_res = {'result mean': counter_list_mean, 
                 'result std': counter_list_std,
                 'time mean': np.mean(time_all_attempts, axis=0),
                 'time std': np.std(time_all_attempts, axis=0),
                 'all data results': qubit_res_all_attempts,
                 'all data times': time_all_attempts}
        
    with open(f"IonQ_{num_qubits}-qubits.json", "wb") as f:
        f.write(json.dumps({num_qubits: final_res}).encode("utf-8"))